In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy # linear algebra
import pandas  # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn
import matplotlib
import sklearn
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import math
from tensorflow import keras
import math
import scipy
state=72
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pandas.read_csv("../input/heart-attack-analysis-prediction-dataset/heart.csv")
#df2 = pandas.read_csv("../input/heart-attack-analysis-prediction-dataset/o2Saturation.csv")

In [ ]:
df.describe(include = "all")

In [ ]:
print(df.isna().sum().sort_values(ascending=False))

In [ ]:
df.shape

In [ ]:
df.head(10)

# Data Analysis

We can see right from the start that are Y variable, the variable we want to be able to predict, is a little bit Skewed. Its a minimal difference but we with later see how this sightly impacts our models

In [ ]:
seaborn.histplot(data = df, x = "output",stat = "count",bins=2,discrete=True)
print("Skewness = {0}".format(df["output"].skew()))

As we can see men are both the major sex in the group of patients with higher prob. of a Heart Attack and in the group of people with lower prob. of Heart Attack, which leads to the conclusion of men being the principal victim of a Heart Attack. Moreover, people in their 50s are very likely to experience some signs of Heart Atack.

In [ ]:
fig, axes = matplotlib.pyplot.subplots(1,3, figsize = (15,5))
matplotlib.pyplot.subplots_adjust(wspace=0.3)


plot1 = seaborn.kdeplot(data = df.loc[df["output"]==0], x = "age",hue="sex", ax = axes [0])
plot1.set_title ("Pacient Age Low Prob")

plot2 = seaborn.kdeplot(data = df.loc[df["output"]==1], x = "age", hue="sex", ax = axes[1])
plot2.set_title ("Pacient Age High Prob")

plot3 = seaborn.histplot (data = df, x="sex",stat="count",ax=axes[2], discrete = True)
plot3.set_title ("Number of People by Sex")

print("Median Age for Patients with Lower Heart Attack Probability: {0:0.2f}".format(df.loc[df["output"]==0]["age"].median()))
print("Median Age for Patients with Higher Heart Attack Probability: {0:0.2f}\n\n".format(df.loc[df["output"]==1]["age"].median()))

Simiarly, patients with higher probability of Heart Attack showed a higher level of chest pain.

In [ ]:
fig, axes = matplotlib.pyplot.subplots(1,2, figsize = (10,5))
plot1 = seaborn.histplot(data = df.loc[df["output"]==0], x = "cp", discrete = True, ax = axes[0])
plot1.set_title("Chest Pain People Low Prob")
plot2 = seaborn.histplot(data = df.loc[df["output"]==1], x = "cp", discrete = True, ax = axes[1])
plot2.set_title("Chest Pain People High Prob")
print("Average Chest Pain for Patients with Low Probability of Heart Attack: {0:0.2f}".format(df.loc[df["output"]==0]["cp"].mean()))
print("Average Chest Pain for Patients with High Probability of Heart Attack: {0:0.2f}\n\n".format(df.loc[df["output"]==1]["cp"].mean()))

The maximum Heart Rate of the Patients also shows promising factor to detect a possible Heart Attack 

In [ ]:
fig, axes = matplotlib.pyplot.subplots(figsize = (8,5))
plot1 = seaborn.kdeplot(data = df, x = "thalachh", hue = "output")
plot1.set_title("Average Heart Rate")
print("Average Heart Rate High Prob. Heart Attack: {0:0.2f}\nAverage Heart Rate Low Prob Heart Attack: {1:0.2f}\n".format(df.loc[df["output"]==1]["thalachh"].mean(),df.loc[df["output"]==0]["thalachh"].mean()))

# Feature Engineering

All the data is numeric. No need to apply any type of Label Enconding

In [ ]:
cat_columns = list()
num_columns = list()
for column in df.columns:
    if(df[column].dtype != "object"):
        num_columns.append(column)
    else:
        cat_columns.append(column)
print(len(cat_columns))

We will plot the Correlation Matrix in order to see any type of Linear Correlation between features. Since this is a Classification problem, most likely there will be low value of Linear Correlation

In [ ]:
corr_data = df.corr()
print(corr_data["output"].sort_values(ascending=False)[1:4])
print(corr_data["output"].sort_values(ascending=True)[:3])
corr_data.style.background_gradient(cmap='coolwarm', axis=None).set_precision(6)

The features here dont show a Linear Correlation with the output variable. With that, choosing the features is a bit harder. 
* We can try to apply PCA and see which principal components have the majority of the variance. Due to the smal number of features, PCA isnt a good option as we will need most of the features to accomplish a variance value of around 90%
* Since we have a small number of features we can simply choose them all and test


In [ ]:
X_columns = df.columns.tolist()
X_columns = X_columns[:-1]
Y_columns = "output"

X = df[X_columns]
Y = df[Y_columns]
std_scaler = preprocessing.StandardScaler()

X_scaled = pandas.DataFrame(std_scaler.fit_transform(X), columns = X_columns)
pca = decomposition.PCA(n_components = 10)
principalComponents = pca.fit_transform(X_scaled)
df_principal = pandas.DataFrame(data = principalComponents
             , columns = ["PC1", "PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"])
print("Total Variance of Principal Components: {0}".format(pca.explained_variance_ratio_.sum()))

A simple Logistic Regression model already shows fairly good results using all the features. We are using a L1 penalty in an atempt to minimize the weights of useless features and usind 3 folds in the Cross-Validation as the test samples is already quite small. Also using Hyperparameter Tuning for the Regularization parameter.

We can see Logistic Regression has a solid F1 Score of 0.8, failling more when predicting False Negatives. Also because there are usually more "1" in the test data than "0"

In [ ]:

X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_scaled, Y, test_size=0.3, random_state=state)


rand_list = {"C": scipy.stats.uniform(2, 30)}

clf = sklearn.linear_model.LogisticRegression (penalty = "l1",solver="saga", random_state=state)

rand_search = sklearn.model_selection.RandomizedSearchCV(clf, param_distributions = rand_list, n_iter = 20, cv = 3, scoring = "f1", random_state=state) 
rand_search.fit(X_train, Y_train) 
print(rand_search.best_params_)

pandas.DataFrame(rand_search.cv_results_).head(5)

In [ ]:
model = sklearn.linear_model.LogisticRegression (penalty = "l2",solver="saga", C = rand_search.best_params_.get("C",20), random_state=state)
model.fit(X_train, Y_train)
print("\nF1 Score: {0:0.2f}\n".format(sklearn.metrics.f1_score(Y_test,model.predict(X_test))))
print(sklearn.metrics.classification_report (Y_test, model.predict(X_test)))
sklearn.metrics.plot_confusion_matrix(model, X_test, Y_test, cmap = matplotlib.pyplot.cm.Blues, normalize = "true")

We can see that the Logistic Regression model considers "Sex" and "Chest Pain" as the most importante features for the problem. That confirms what we saw earlier.

In [ ]:
fig, axes = matplotlib.pyplot.subplots(1, figsize = (10,8))
for i in range(0,13):
    print("Feature: {0} - {1:0.4f}".format(X_scaled.columns.tolist()[i],model.coef_[0][i]))
plot1 = seaborn.barplot(x = X_train.columns.tolist(),y = model.coef_[0], palette=['red' if (x > 0.7 or x < -0.7) else 'blue' for x in model.coef_[0]], ax = axes)

#plot1.set_xticklabels(X_train.columns)


Let's compare it to a Random Florest model

This model, with Hyperparameter Tuning, has an increse of 0.02, average, in the F1 Score in comparision with Logistic Regression.There is still the problem of the model predicting more False Negatives than False Positves due to higher number of samples of Class "1"

In [ ]:
from sklearn import ensemble

rand_list = {
            "max_depth":[None,1,2],
            "min_samples_split":[2,3,4]}

clf = ensemble.RandomForestClassifier(random_state=state)
#f1 = sklearn.metrics.make_scorer(sklearn.metrics.f1_score)
rand_search = sklearn.model_selection.RandomizedSearchCV(clf, param_distributions = rand_list, n_iter = 20, cv = 3, scoring = "f1",return_train_score=True, error_score = "raise", random_state=state) 
rand_search.fit(X_train, Y_train) 
print(rand_search.best_params_)


pandas.DataFrame(rand_search.cv_results_).head(5)

In [ ]:
model2 = ensemble.RandomForestClassifier(max_depth = rand_search.best_params_.get("max_depth"), min_samples_split = rand_search.best_params_.get("min_samples_split"), random_state=state)
model2.fit(X_train,Y_train)
print("\nF1 Score: {0:0.2f}\n".format(sklearn.metrics.f1_score(Y_test,model2.predict(X_test))))
print(sklearn.metrics.classification_report (Y_test, model2.predict(X_test)))
sklearn.metrics.plot_confusion_matrix(model2, X_test, Y_test,cmap = matplotlib.pyplot.cm.Blues, normalize = "true")

We can see that both Logistic Regression and Random Florest have similiar Weight's importance. However the Random Florest Model determines that "maximum heart rate achieved" and the "number of major vessels" are also very important features.

In [ ]:
fig, axes = matplotlib.pyplot.subplots(1, figsize = (10,6))
for i in range(0,13):
    print("Feature: {0} - {1:0.4f}".format(X_scaled.columns.tolist()[i],model2.feature_importances_[i]))
plot1 = seaborn.barplot(x = X_train.columns.tolist(),y = model2.feature_importances_, palette=['red' if (x > 0.125) else 'blue' for x in model2.feature_importances_])

#plot1.set_xticklabels(X_train.columns)

Let's try one last model - K Nearest Neighbors

A K Nearest Neighbors with Hyperparameter Tuning has a lower F1 score, of up to 0.84.

In [ ]:
rand_list = {
            "n_neighbors":[5,6,7,8,9,10],
            "algorithm":["ball_tree","kd_tree","brute"]}

clf = sklearn.neighbors.KNeighborsClassifier()

rand_search = sklearn.model_selection.RandomizedSearchCV(clf, param_distributions = rand_list, n_iter = 20, cv = 3, scoring = "f1",return_train_score=True, error_score = "raise",random_state=state) 
rand_search.fit(X_train, Y_train) 
print(rand_search.best_params_)

pandas.DataFrame(rand_search.cv_results_).head(5)


In [ ]:
model3 = sklearn.neighbors.KNeighborsClassifier(n_neighbors = rand_search.best_params_.get("n_neighbors"), algorithm = rand_search.best_params_.get("algorithm"))
model3.fit(X_train,Y_train)
print("\nF1 Score: {0:0.2f}\n".format(sklearn.metrics.f1_score(Y_test,model3.predict(X_test))))
print(sklearn.metrics.classification_report (Y_test, model3.predict(X_test)))
sklearn.metrics.plot_confusion_matrix(model3, X_test, Y_test,cmap = matplotlib.pyplot.cm.Blues, normalize = "true")

In [ ]:
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=numpy.linspace(.1, 1.0, 5), scoring = None):
    
    
    if axes is None:
        _, axes = matplotlib.pyplot.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True, scoring = scoring)
        
        
        
    train_scores_mean = numpy.mean(train_scores, axis=1)
    train_scores_std = numpy.std(train_scores, axis=1)
    test_scores_mean = numpy.mean(test_scores, axis=1)
    test_scores_std = numpy.std(test_scores, axis=1)
    fit_times_mean = numpy.mean(fit_times, axis=1)
    fit_times_std = numpy.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")
    

    return matplotlib.pyplot

The Random Florest model shows a higher score for this dataset. However, Logistic Regression shows a higher scalability to a much larger dataset. In this case we are only conserned with the score of the model. K Nearest Neighbors does have a higher scalability than Random Florest and it does seem to tend to a similar score, however more samples are needed to confirm this

In [ ]:
fig, axes = matplotlib.pyplot.subplots(3,3,figsize = (16,12))
plot_learning_curve(model, "Learning Curves Logistic", X_scaled, Y, axes=axes[:, 0], ylim=(0.7, 1.01),n_jobs=4, scoring="f1")
matplotlib.pyplot.subplots_adjust(hspace=0.4)

plot_learning_curve(model2, "Learning Curves Random Florest", X_scaled, Y, axes=axes[:, 1], ylim=(0.7, 1.01),n_jobs=4, scoring="f1")
matplotlib.pyplot.subplots_adjust(hspace=0.4)

plot_learning_curve(model3, "Learning Curves K Neighbors", X_scaled, Y, axes=axes[:, 2], ylim=(0.7, 1.01),n_jobs=4, scoring="f1")
matplotlib.pyplot.subplots_adjust(hspace=0.4)

Due to the higher score, the model that better suits this Dataset is Random Florest!